In [ ]:
# convert form CIF to pdb
!obabel ../molecules/fold_2025_08_04_12_53-\ HIV-1-Protease/fold_2025_08_04_12_53_model_0.cif  -opdb -O ../molecules/HIV-1-Protease-AlphaFold.pdb

In [ ]:
## saves in the same folder as the molecule so make sure you transfer later - Pocket finder
! /Users/kap037/Desktop/CSIRO-Malaysia-alphafold/scripts/fpocket/bin/fpocket -f ../protein-structure-prediction/fold_wt_hiv1_protease_dimer/fold_wt_hiv1_protease_dimer_model_0.cif

### Importing essential utilities 

In [ ]:
import subprocess
from pathlib import Path
import numpy as np

from rdkit import rdBase
rdBase.DisableLog('rdApp.*')

from vina import Vina
from rdkit import Chem
from pdbfixer import PDBFixer
from openmm.app import PDBFile
import py3Dmol

def run_command(command):
    print(f"Running: {command}")
    proc = subprocess.run(command, shell=True, capture_output=True, text=True)
    if proc.returncode != 0:
        print(proc.stderr)
        raise RuntimeError(f"Command failed: {command}")
    else:
        print(proc.stdout)

def protonate_protein(pdb_file, output_pdb_file, ph=7.4):
    fixer = PDBFixer(filename=str(pdb_file))
    fixer.removeHeterogens(keepWater=False)
    fixer.findMissingResidues()
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(pH=ph)
    with open(str(output_pdb_file), 'w') as out_f:
        PDBFile.writeFile(fixer.topology, fixer.positions, out_f)
    print(f"Protein protonated at pH {ph} and saved to {output_pdb_file}")

def extract_ligand_only(pdb_file, ligand_pdb_file, ligand_resname):
    """Extract ONLY ligand atoms with strict validation to avoid protein contamination"""
    ligand_lines = []
    protein_residues = {'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 
                       'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'}
    
    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith(('ATOM', 'HETATM')):
                resname = line[17:20].strip()
                if resname == ligand_resname:
                    # Double-check this isn't a protein residue
                    if resname not in protein_residues:
                        ligand_lines.append(line)
    
    if not ligand_lines:
        raise ValueError(f"No ligand atoms found for residue '{ligand_resname}'")
    
    # Write ligand-only PDB
    with open(ligand_pdb_file, 'w') as f:
        f.writelines(ligand_lines)
        f.write("END\n")
    
    print(f"Extracted {len(ligand_lines)} pure ligand atoms to {ligand_pdb_file}")

def prepare_protein_openbabel(protein_pdb_path, protein_pdbqt_path):
    """Use Open Babel to prepare protein PDBQT and clean inappropriate tags"""
    run_command(f"obabel {protein_pdb_path} -O {protein_pdbqt_path}")
    
    # Clean the PDBQT file by removing ligand-specific tags
    with open(protein_pdbqt_path, 'r') as f:
        lines = f.readlines()
    
    cleaned_lines = []
    skip_section = False
    
    for line in lines:
        line_stripped = line.strip()
        
        if line_stripped.startswith('ROOT'):
            skip_section = True
            continue
        elif line_stripped.startswith('ENDROOT'):
            skip_section = False
            continue
        elif line_stripped.startswith(('BRANCH', 'ENDBRANCH', 'TORSDOF')):
            continue
        
        if not skip_section:
            cleaned_lines.append(line)
    
    with open(protein_pdbqt_path, 'w') as f:
        f.writelines(cleaned_lines)
    
    print(f"Protein PDBQT prepared and cleaned: {protein_pdbqt_path}")

def prepare_ligand_openbabel(ligand_pdb_path, ligand_pdbqt_path, ph=7.4):
    """Use Open Babel to prepare ligand PDBQT with validation"""
    run_command(f"obabel {ligand_pdb_path} -O {ligand_pdbqt_path} -p {ph}")
    
    # Validate that ligand PDBQT doesn't contain protein residues
    protein_residues = {'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 
                       'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'}
    
    with open(ligand_pdbqt_path, 'r') as f:
        content = f.read()
        for residue in protein_residues:
            if residue in content:
                lines = content.split('\n')
                for line_num, line in enumerate(lines, 1):
                    if line.startswith(('ATOM', 'HETATM')) and residue in line:
                        raise ValueError(f"Protein residue {residue} found in ligand PDBQT at line {line_num}")
    
    print(f"Ligand PDBQT prepared and validated: {ligand_pdbqt_path}")

def get_ligand_center(ligand_pdb_path):
    mol = Chem.MolFromPDBFile(str(ligand_pdb_path), removeHs=False)
    if mol is None:
        raise ValueError(f"Could not load ligand from {ligand_pdb_path}")
    conf = mol.GetConformer()
    coords = np.array(conf.GetPositions())
    #print(coords)
    center = coords.mean(axis=0)
    print(f"Ligand geometric center: {center.tolist()}")
    return center.tolist()

def visualize_complex(protein_pdb_path, ligand_pdb_path, box_center, box_size):
    if isinstance(box_size, (int, float)):
        box_size = [box_size] * 3 #size factor <3 needs dimension setting explicitly
    
    view = py3Dmol.view(width=800, height=600)
    
    # Add protein dimer
    with open(protein_pdb_path) as f:
        protein_str = f.read()
    view.addModel(protein_str, "pdb")
    view.setStyle({'model': 0}, {'cartoon': {'color': 'spectrum'}})
    
    # Add ligand
    with open(ligand_pdb_path) as f:
        ligand_str = f.read()
    view.addModel(ligand_str, "pdb")
    view.setStyle({'model': 1}, {'stick': {'colorscheme': 'cyanCarbon'}})
    
    # Add docking box
    x, y, z = box_center
    view.addBox({
        'center': {'x': x, 'y': y, 'z': z},
        'dimensions': {'w': box_size[0], 'h': box_size[1], 'd': box_size[2]},
        'color': 'red',
        'opacity': 0.8,
        'wireframe': True,
        'linewidth': 3
    })
    
    view.zoomTo()
    view.show()
    #view.exportImage("/Users/kap037/Desktop/CSIRO-Malaysia-alphafold/molecules/docking/syneromics/docking_box.png", { 'Quality': 1.0, 'width': 800, 'height': 600 } )

## Binding scoring workflow with AutoDock VINA

In [ ]:
def score_and_visualize_alphafold_complex(
    cif_file,
    ligand_resname,
    pH=7.4,
    box_size=20.0
):
    base = Path(cif_file).stem
    pdb_file = Path(f"{base}.pdb")
    protein_pdb = Path(f"{base}_protein.pdb")
    ligand_pdb = Path(f"{base}_ligand.pdb")
    protein_ph_pdb = Path(f"{base}_protein_ph_{pH}.pdb")
    protein_pdbqt = Path(f"{base}_protein.pdbqt")
    ligand_pdbqt = Path(f"{base}_ligand.pdbqt")

    print(f"Processing homo-dimer complex '{Path(cif_file).name}' with ligand '{ligand_resname}' at pH {pH}")

    # Step 1: Convert CIF to PDB
    run_command(f"obabel {cif_file} -O {pdb_file}")

    # Step 2: Split protein and ligand
    run_command(f"obabel {pdb_file} -O {protein_pdb} --delete resname {ligand_resname},HOH")
    #run_command(f"obabel {pdb_file} -O {protein_pdb} --delete resname HOH")
    
    
    extract_ligand_only(pdb_file, ligand_pdb, ligand_resname)  # Use validated extraction

    # Step 3: Protonate the dimer at specified pH
    protonate_protein(protein_pdb, protein_ph_pdb, pH)

    # Step 4: Prepare protein PDBQT
    prepare_protein_openbabel(protein_ph_pdb, protein_pdbqt)

    # Step 5: Prepare ligand PDBQT
    prepare_ligand_openbabel(ligand_pdb, ligand_pdbqt, pH)

    # Step 6: Calculate ligand center for docking box
    centerr = get_ligand_center(ligand_pdb)
    print(centerr)


    #Step 7: Score binding affinity with AutoDock Vina
    v = Vina(sf_name='vina')
    v.set_receptor(str(protein_pdbqt))
    v.set_ligand_from_file(str(ligand_pdbqt))
    box_dims = [box_size] * 3
    #center = [2.693714286, -8.111809524, 2.676071429]
    v.compute_vina_maps(center=centerr, box_size=box_dims)
    
    # Raw score of current pose
    energy_raw = v.score()
    print(f"Raw pose score: {energy_raw[0]:.3f} kcal/mol")

    # Local minimisation of current pose, then score
    energy_min = v.optimize()
    print(f"Predicted binding affinity after local minimization: {energy_min[0]:.3f} kcal/mol")


    # Step 8: Visualize the complex
    print("Generating 3D visualization...")
    box_dims = [box_size] * 3
    visualize_complex(protein_ph_pdb, ligand_pdb, centerr, box_dims)


    return float(energy_min[0])

## Run program

In [ ]:
# Alphafold3 HIV-1 protease dimer + darunavir analysis
cif_file_path = '/Users/kap037/Desktop/CSIRO-Malaysia-alphafold/molecules/docking/in-house-alphafold3-dimer_mut_darunavir/'
ligand_res_name = 'LIG_B'
target_pH = 5.5 #7.4 #https://www.ncbi.nlm.nih.gov/books/NBK507807/ for physiological ph 
docking_box_size = 20.0

score = score_and_visualize_alphafold_complex(
    cif_file=cif_file_path,
    ligand_resname=ligand_res_name,
    pH=target_pH,
    box_size=docking_box_size
)

print(f"Final predicted binding affinity: {score:.3f} kcal/mol")
